In [2]:
# Lab 6 Softmax Classifier
import tensorflow as tf
import numpy as np
tf.set_random_seed(777)  # for reproducibility

C:\Users\sds\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
# Predicting animal type based on various features
xy = np.loadtxt('data-04-zoo.csv', delimiter=',', dtype=np.float32)
x_data = xy[:, 0:-1]  
y_data = xy[:, [-1]]

print(x_data.shape, y_data.shape)

'''
(101, 16) (101, 1)
'''

(101, 16) (101, 1)


'\n(101, 16) (101, 1)\n'

In [4]:
xy # 0~15개의 16개 열은 X변수, # 마지막 열은 Y변수(label)

array([[1., 0., 0., ..., 0., 1., 0.],
       [1., 0., 0., ..., 0., 1., 0.],
       [0., 0., 1., ..., 0., 0., 3.],
       ...,
       [1., 0., 0., ..., 0., 1., 0.],
       [0., 0., 1., ..., 0., 0., 6.],
       [0., 1., 1., ..., 0., 0., 1.]], dtype=float32)

In [5]:
3

nb_classes = 7  # 0 ~ 6

X = tf.placeholder(tf.float32, [None, 16])
Y = tf.placeholder(tf.int32, [None, 1])  # 0 ~ 6

In [6]:
Y_one_hot = tf.one_hot(Y, nb_classes)  # one hot
print("one_hot:", Y_one_hot)
Y_one_hot = tf.reshape(Y_one_hot, [-1, nb_classes])
print("reshape one_hot:", Y_one_hot)

'''
one_hot: Tensor("one_hot:0", shape=(?, 1, 7), dtype=float32)
reshape one_hot: Tensor("Reshape:0", shape=(?, 7), dtype=float32)
'''

one_hot: Tensor("one_hot:0", shape=(?, 1, 7), dtype=float32)
reshape one_hot: Tensor("Reshape:0", shape=(?, 7), dtype=float32)


'\none_hot: Tensor("one_hot:0", shape=(?, 1, 7), dtype=float32)\nreshape one_hot: Tensor("Reshape:0", shape=(?, 7), dtype=float32)\n'

In [7]:
Y

<tf.Tensor 'Placeholder_1:0' shape=(?, 1) dtype=int32>

In [8]:
Y_one_hot

<tf.Tensor 'Reshape:0' shape=(?, 7) dtype=float32>

In [9]:
W = tf.Variable(tf.random_normal([16, nb_classes]), name='weight')
b = tf.Variable(tf.random_normal([nb_classes]), name='bias')

# tf.nn.softmax computes softmax activations
# softmax = exp(logits) / reduce_sum(exp(logits), dim)
logits = tf.matmul(X, W) + b
hypothesis = tf.nn.softmax(logits)

# Cross entropy cost/loss # 함수 이름이 좀 길다 # v2로 업데이트된듯
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits,
                                                                 labels=tf.stop_gradient([Y_one_hot])))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

In [10]:
# 학습
prediction = tf.argmax(hypothesis, 1) # argmax를 취해서 hypothesis 를 label로 바꾼다, probability를 0~6사이의 값으로
correct_prediction = tf.equal(prediction, tf.argmax(Y_one_hot, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [16]:
# Launch graph
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for step in range(2001):
        _, cost_val, acc_val = sess.run([optimizer, cost, accuracy], feed_dict={X: x_data, Y: y_data})
                                        
        if step % 100 == 0:
            print("Step: {:5}\tCost: {:.3f}\tAcc: {:.2%}".format(step, cost_val, acc_val))
            
    # 학습이 다 끝나면
    print('--- 학습이 다 끝나면 see if we can predict ---')
    # Let's see if we can predict
    pred = sess.run(prediction, feed_dict={X: x_data})
    # y_data: (N,1) = flatten => (N, ) matches pred.shape
    for p, y in zip(pred, y_data.flatten()):
        print("[{}] Prediction: {} True Y: {}".format(p == int(y), p, int(y)))

'''
Step:     0 Loss: 5.106 Acc: 37.62%
Step:   100 Loss: 0.800 Acc: 79.21%
Step:   200 Loss: 0.486 Acc: 88.12%
...
Step:  1800	Loss: 0.060	Acc: 100.00%
Step:  1900	Loss: 0.057	Acc: 100.00%
Step:  2000	Loss: 0.054	Acc: 100.00%
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 3 True Y: 3
...
[True] Prediction: 0 True Y: 0
[True] Prediction: 6 True Y: 6
[True] Prediction: 1 True Y: 1
'''

Step:     0	Cost: 5.480	Acc: 37.62%
Step:   100	Cost: 0.806	Acc: 79.21%
Step:   200	Cost: 0.488	Acc: 88.12%
Step:   300	Cost: 0.350	Acc: 90.10%
Step:   400	Cost: 0.272	Acc: 94.06%
Step:   500	Cost: 0.222	Acc: 95.05%
Step:   600	Cost: 0.187	Acc: 97.03%
Step:   700	Cost: 0.161	Acc: 97.03%
Step:   800	Cost: 0.141	Acc: 97.03%
Step:   900	Cost: 0.124	Acc: 97.03%
Step:  1000	Cost: 0.111	Acc: 97.03%
Step:  1100	Cost: 0.101	Acc: 99.01%
Step:  1200	Cost: 0.092	Acc: 100.00%
Step:  1300	Cost: 0.084	Acc: 100.00%
Step:  1400	Cost: 0.078	Acc: 100.00%
Step:  1500	Cost: 0.072	Acc: 100.00%
Step:  1600	Cost: 0.068	Acc: 100.00%
Step:  1700	Cost: 0.064	Acc: 100.00%
Step:  1800	Cost: 0.060	Acc: 100.00%
Step:  1900	Cost: 0.057	Acc: 100.00%
Step:  2000	Cost: 0.054	Acc: 100.00%
--- 학습이 다 끝나면 see if we can predict ---
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 3 True Y: 3
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Pr

'\nStep:     0 Loss: 5.106 Acc: 37.62%\nStep:   100 Loss: 0.800 Acc: 79.21%\nStep:   200 Loss: 0.486 Acc: 88.12%\n...\nStep:  1800\tLoss: 0.060\tAcc: 100.00%\nStep:  1900\tLoss: 0.057\tAcc: 100.00%\nStep:  2000\tLoss: 0.054\tAcc: 100.00%\n[True] Prediction: 0 True Y: 0\n[True] Prediction: 0 True Y: 0\n[True] Prediction: 3 True Y: 3\n...\n[True] Prediction: 0 True Y: 0\n[True] Prediction: 6 True Y: 6\n[True] Prediction: 1 True Y: 1\n'